In [1]:
#!pip install transformers
#!pip install datasets
#!pip install ekphrasis
#!pip install spacy
#!python -m spacy download en_core_web_sm

## Load in dataset 

In [1]:
import pandas as pd

selected_examples = pd.read_csv('selected_examples_for_paper.csv')

## Implementation of EDA: Easy Data Augmentation Techniques for Boosting Performance on Text Classification Tasks

In [2]:
import nltk
import random
from random import shuffle
nltk.download('wordnet')
from nltk.corpus import wordnet 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\evanc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Easy data augmentation techniques for text classification
# Jason Wei and Kai Zou

random.seed(2022)

#stop words list
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
			'ours', 'ourselves', 'you', 'your', 'yours', 
			'yourself', 'yourselves', 'he', 'him', 'his', 
			'himself', 'she', 'her', 'hers', 'herself', 
			'it', 'its', 'itself', 'they', 'them', 'their', 
			'theirs', 'themselves', 'what', 'which', 'who', 
			'whom', 'this', 'that', 'these', 'those', 'am', 
			'is', 'are', 'was', 'were', 'be', 'been', 'being', 
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at', 
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after', 
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again', 
			'further', 'then', 'once', 'here', 'there', 'when', 
			'where', 'why', 'how', 'all', 'any', 'both', 'each', 
			'few', 'more', 'most', 'other', 'some', 'such', 'no', 
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
			'very', 's', 't', 'can', 'will', 'just', 'don', 
			'should', 'now', '']

#cleaning up text
import re
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line

########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	#this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word): 
		for l in syn.lemmas(): 
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym) 
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################

def random_deletion(words, p):

	#obviously, if there's only one word, don't delete it
	if len(words) == 1:
		return words

	#randomly delete words with probability p
	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	#if you end up deleting all words, just return a random word
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################

def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		random_word = new_words[random.randint(0, len(new_words)-1)]
		synonyms = get_synonyms(random_word)
		counter += 1
		if counter >= 10:
			return
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

########################################################################
# main data augmentation function
########################################################################

def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	
	sentence = get_only_chars(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not '']
	num_words = len(words)
	
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4)+1

	#sr
	if (alpha_sr > 0):
		n_sr = max(1, int(alpha_sr*num_words))
		for _ in range(num_new_per_technique):
			a_words = synonym_replacement(words, n_sr)
			augmented_sentences.append(' '.join(a_words))

	#ri
	if (alpha_ri > 0):
		n_ri = max(1, int(alpha_ri*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_insertion(words, n_ri)
			augmented_sentences.append(' '.join(a_words))

	#rs
	if (alpha_rs > 0):
		n_rs = max(1, int(alpha_rs*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_swap(words, n_rs)
			augmented_sentences.append(' '.join(a_words))

	#rd
	if (p_rd > 0):
		for _ in range(num_new_per_technique):
			a_words = random_deletion(words, p_rd)
			augmented_sentences.append(' '.join(a_words))

	augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
	shuffle(augmented_sentences)

	#trim so that we have the desired number of augmented sentences
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	#append the original sentence
	augmented_sentences.append(sentence)

	return augmented_sentences

<>:170: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:170: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\evanc\AppData\Local\Temp/ipykernel_28540/625384722.py:170: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not '']


In [5]:
selected_examples.head()


,Unnamed: 0,post_id,target1,target2,target3,final_label,text_combined
0,7839,5529520_gab,"['Islam', 'Refugee']",['Islam'],['Islam'],hatespeech,fuck trudeau no to islam kick the muzzies out ...
1,8442,11762516_gab,['Other'],['None'],['None'],offensive,go back to breitbart you sound like a retarded...
2,8956,1182326476101836800_twitter,['Other'],['None'],['Other'],offensive,gentle reminder that game journalists are boot...
3,9167,1178607326833577985_twitter,['African'],['African'],['None'],hatespeech,you dumb nigger im gonna have your head on a p...
4,9540,1179097350519828480_twitter,['Caucasian'],['Caucasian'],['Economic'],offensive,do not ever buy a car from a redneck


In [9]:
selected_examples.rename({'text_combined' : 'text_str'}, axis=1, inplace=True)

selected_examples.head()

,Unnamed: 0,post_id,target1,target2,target3,final_label,text_str
0,7839,5529520_gab,"['Islam', 'Refugee']",['Islam'],['Islam'],hatespeech,fuck trudeau no to islam kick the muzzies out ...
1,8442,11762516_gab,['Other'],['None'],['None'],offensive,go back to breitbart you sound like a retarded...
2,8956,1182326476101836800_twitter,['Other'],['None'],['Other'],offensive,gentle reminder that game journalists are boot...
3,9167,1178607326833577985_twitter,['African'],['African'],['None'],hatespeech,you dumb nigger im gonna have your head on a p...
4,9540,1179097350519828480_twitter,['Caucasian'],['Caucasian'],['Economic'],offensive,do not ever buy a car from a redneck


In [11]:
#######################################
# Set initial parameters to run EDA on data
# run all cells below to create dataframe for each set of parameters
#######################################

def initialize_params(num_aug=9, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, alpha_rd=0.1):
    # num_aug: number of augmented sentences to generate per original sentence
    # alpha_sr: how much to replace each word by synonyms
    # alpha_ri: how much to insert new words that are synonyms
    # alpha_rs: how much to swap words
    # alpha_rd: how much to delete words
    return num_aug, alpha_sr, alpha_ri, alpha_rs, alpha_rd

In [12]:
def run_EDA(raw_data, num_aug, alpha_sr, alpha_ri, alpha_rs, alpha_rd):
    # initialize a dataframe for output
    df_EDA = pd.DataFrame(columns = ['post_id', 'text_str', 'final_label'])

    # Run EDA for all the rows in raw data 
    for i in range(len(raw_data)):
        #generate more data with standard augmentation
        aug_sentences = eda(raw_data.loc[i,'text_str'] , 
                            alpha_sr=alpha_sr, alpha_ri=alpha_ri, alpha_rs=alpha_rs, p_rd=alpha_rd, num_aug=num_aug)
        post_id = [raw_data.loc[i,'post_id'] ] * (num_aug+1)
        final_label = [raw_data.loc[i,'final_label'] ] * (num_aug+1)

        df_append = pd.DataFrame({"post_id": post_id,
                                  "text_str": aug_sentences, 
                                  "final_label": final_label})

        # append to EDA dataframe
        df_EDA = df_EDA.append(df_append, ignore_index = True)
        
    return df_EDA

In [25]:
# alpha_sr: how much to replace each word by synonyms
df_EDA = run_EDA(selected_examples, 5, 0.7,0.01, 0.01, 0.01)
df_EDA.to_csv('./EDA_5_0_7_sr_rest_0_1.csv')

C:\Users\evanc\AppData\Local\Temp/ipykernel_28540/2397526769.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_EDA = df_EDA.append(df_append, ignore_index = True)


In [26]:
# alpha_ri: how much to insert new words that are synonyms
df_EDA = run_EDA(selected_examples, 5, 0.01,0.7,0.01,0.01)
df_EDA.to_csv('./EDA_5_0_7_ri_rest_0_1.csv')

C:\Users\evanc\AppData\Local\Temp/ipykernel_28540/2397526769.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_EDA = df_EDA.append(df_append, ignore_index = True)


In [27]:
# alpha_rs: how much to swap words
df_EDA = run_EDA(selected_examples, 5, 0.01,0.01,0.7,0.01)
df_EDA.to_csv('./EDA_5_0_7_rs_rest_0_1.csv')

C:\Users\evanc\AppData\Local\Temp/ipykernel_28540/2397526769.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_EDA = df_EDA.append(df_append, ignore_index = True)


In [28]:
# alpha_rd: how much to delete words
df_EDA = run_EDA(selected_examples, 5, 0.01,0.01,0.01,0.7)
df_EDA.to_csv('./EDA_5_0_7_rd_rest_0_1.csv')

C:\Users\evanc\AppData\Local\Temp/ipykernel_28540/2397526769.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_EDA = df_EDA.append(df_append, ignore_index = True)
